In [1]:
import pandas as pd
import requests
import gmaps
import json
import numpy as np
from citipy import citipy
import time
from datetime import datetime
from config import weather_api_key
url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key

In [22]:
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-180.000, 180.000, size=2000)
lat_lng = zip(lats, lngs)
lat_lng

In [23]:
coordinates = list(lat_lng)

In [24]:
cities = []

# use nearest_city to find the nearest city to the coordinates
for coordinate in coordinates:
        city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
        
        if city not in cities:
            cities.append(city)
            
len(cities)    

747

In [25]:
city_data = []

print("Beginning data retrieval       ")
print("-------------------------------")

# create counters
record_count = 1
set_count = 1

for i, city in enumerate(cities):

    # group cities in sets of 50 for logging purposes
    if(i % 50 == 0 and i >= 50):
            set_count += 1
            record_count = 1
            time.sleep(60)

    
    city_url = url + "&q=" + city.replace(" ", "+")

    
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    record_count += 1

    
    try:
        
        city_weather = requests.get(city_url).json()
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind= city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        current_description = city_weather["weather"][0]["description"]
        
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
       
        city_weather_stats = {"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Country": city_country,
                         "Date": city_date,
                         "Current Description": current_description}
        city_data.append(city_weather_stats)
        

   
    except:
        print("City not found. Skipping...")
    pass


print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning data retrieval       
-------------------------------
Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | kosh-agach
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | charters towers
Processing Record 5 of Set 1 | bredasdorp
Processing Record 6 of Set 1 | talnakh
Processing Record 7 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 8 of Set 1 | castro
Processing Record 9 of Set 1 | kodiak
Processing Record 10 of Set 1 | fort nelson
Processing Record 11 of Set 1 | porto seguro
Processing Record 12 of Set 1 | zhuanghe
Processing Record 13 of Set 1 | nikolskoye
Processing Record 14 of Set 1 | cape town
Processing Record 15 of Set 1 | hilo
Processing Record 16 of Set 1 | svetlogorsk
Processing Record 17 of Set 1 | olinda
Processing Record 18 of Set 1 | saskylakh
Processing Record 19 of Set 1 | gargalianoi
Processing Record 20 of Set 1 | vao
Processing Record 21 of Set 1 | jamestown
Processing Record 22 of Set 1 | ma

In [34]:
len(city_data)

682

In [35]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Punta Arenas,-53.1500,-70.9167,51.91,66,75,19.57,CL,2022-04-20 17:30:17,broken clouds
1,Kosh-Agach,50.0000,88.6667,33.64,51,94,8.28,RU,2022-04-20 17:32:34,overcast clouds
2,Busselton,-33.6500,115.3333,54.95,85,0,11.90,AU,2022-04-20 17:32:35,clear sky
3,Charters Towers,-20.1000,146.2667,70.50,76,85,6.08,AU,2022-04-20 17:32:35,overcast clouds
4,Bredasdorp,-34.5322,20.0403,70.14,45,62,5.26,ZA,2022-04-20 17:28:47,broken clouds
...,...,...,...,...,...,...,...,...,...,...
677,Cotonou,6.3654,2.4183,85.98,74,20,16.11,BJ,2022-04-20 17:52:52,few clouds
678,Asmara,15.3333,38.9333,65.93,58,32,6.24,ER,2022-04-20 17:52:53,scattered clouds
679,Panacan,9.2463,118.4082,80.37,83,100,5.75,PH,2022-04-20 17:52:53,overcast clouds
680,Olafsvik,64.8945,-23.7142,44.60,88,100,15.64,IS,2022-04-20 17:52:54,overcast clouds


In [36]:
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Date', 'Current Description']
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,51.91,66,75,19.57,2022-04-20 17:30:17,broken clouds
1,Kosh-Agach,RU,50.0000,88.6667,33.64,51,94,8.28,2022-04-20 17:32:34,overcast clouds
2,Busselton,AU,-33.6500,115.3333,54.95,85,0,11.90,2022-04-20 17:32:35,clear sky
3,Charters Towers,AU,-20.1000,146.2667,70.50,76,85,6.08,2022-04-20 17:32:35,overcast clouds
4,Bredasdorp,ZA,-34.5322,20.0403,70.14,45,62,5.26,2022-04-20 17:28:47,broken clouds
...,...,...,...,...,...,...,...,...,...,...
677,Cotonou,BJ,6.3654,2.4183,85.98,74,20,16.11,2022-04-20 17:52:52,few clouds
678,Asmara,ER,15.3333,38.9333,65.93,58,32,6.24,2022-04-20 17:52:53,scattered clouds
679,Panacan,PH,9.2463,118.4082,80.37,83,100,5.75,2022-04-20 17:52:53,overcast clouds
680,Olafsvik,IS,64.8945,-23.7142,44.60,88,100,15.64,2022-04-20 17:52:54,overcast clouds


In [38]:
output_data_file = '/Users/adambachrach/Desktop/data-bootcamp/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv'

city_data_df.to_csv(output_data_file)